In [2]:
a = list(range(10))
a.pop(3)   # a = [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

3

In [3]:
import time

In [4]:
def load_embeddings(CorpusPretrainedEmbdDict,word2idx_vocab, word2idx_embed,embedding_dim):
    embeddings=np.zeros((len(word2idx_vocab),embedding_dim))
    for word,index in word2idx_vocab.items():
        vector = np.array(CorpusPretrainedEmbdDict[word], dtype='float32')
        embeddings[index]=vector
    return embeddings

In [29]:
from torchtext.vocab import Vectors
import torch.nn as nn
import torch
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

vectors = Vectors(name='glove.6B.300d.txt', cache='/data/WordEmbeddingPre-trained/Glove/glove.6B/')

#Create an embedding layer
embedder= nn.Embedding(len(vectors.stoi),vectors.dim).to(device)
#Feed our pretrained word vectors to embedding layer
embedder.weight.data.copy_(vectors.vectors)

NameError: name 'get_tokenizer' is not defined

In [8]:
vectors.stoi

{'the': 0,
 ',': 1,
 '.': 2,
 'of': 3,
 'to': 4,
 'and': 5,
 'in': 6,
 'a': 7,
 '"': 8,
 "'s": 9,
 'for': 10,
 '-': 11,
 'that': 12,
 'on': 13,
 'is': 14,
 'was': 15,
 'said': 16,
 'with': 17,
 'he': 18,
 'as': 19,
 'it': 20,
 'by': 21,
 'at': 22,
 '(': 23,
 ')': 24,
 'from': 25,
 'his': 26,
 "''": 27,
 '``': 28,
 'an': 29,
 'be': 30,
 'has': 31,
 'are': 32,
 'have': 33,
 'but': 34,
 'were': 35,
 'not': 36,
 'this': 37,
 'who': 38,
 'they': 39,
 'had': 40,
 'i': 41,
 'which': 42,
 'will': 43,
 'their': 44,
 ':': 45,
 'or': 46,
 'its': 47,
 'one': 48,
 'after': 49,
 'new': 50,
 'been': 51,
 'also': 52,
 'we': 53,
 'would': 54,
 'two': 55,
 'more': 56,
 "'": 57,
 'first': 58,
 'about': 59,
 'up': 60,
 'when': 61,
 'year': 62,
 'there': 63,
 'all': 64,
 '--': 65,
 'out': 66,
 'she': 67,
 'other': 68,
 'people': 69,
 "n't": 70,
 'her': 71,
 'percent': 72,
 'than': 73,
 'over': 74,
 'into': 75,
 'last': 76,
 'some': 77,
 'government': 78,
 'time': 79,
 '$': 80,
 'you': 81,
 'years': 82,
 'i

In [11]:
class Embedder():
    def __init__(self, embedder_name, embedder_path, device):
        self.embedder_name = embedder_name
        self.embedder_path = embedder_path
        self.vectors = Vectors(name=embedder_name, cache=embedder_path)
        self.embedder = nn.Embedding(len(vectors.stoi),vectors.dim).to(device)
        self.embedder.weight.data.copy_(vectors.vectors)
        self.unknow = 0 # self.vectors.stoi['<unk>']
        
    def embed(self, x):
        return self.embedder(x).to(device)
        
    def one_hot_encode(self, sentence):
        one_hot = []
        for word in sentence:
            if word in self.vectors.stoi :
                one_hot.append(self.vectors.stoi[word])
            else :
                one_hot.append(self.unknow)
        return torch.tensor(one_hot).to(device)
    
    def tensorize(self, x):
        return self.embed(self.one_hot_encode(x)).to(device)

In [12]:
emb = Embedder('glove.6B.300d.txt', '/data/WordEmbeddingPre-trained/Glove/glove.6B/', device)

In [13]:
emb.tensorize(['<sos>','love']).shape

torch.Size([2, 300])

In [14]:
import torch
from torch.utils import data
import pandas as pd

In [15]:
import torchtext
import spacy


In [16]:
class AllSentencesDataset(data.Dataset):
    def __init__(self, path, embedder):
        self.path = path
        with open(self.path) as f:
            self.len =  sum(1 for line in f)
        self.embedder = embedder
        TEXT = torchtext.data.Field(tokenize='spacy',
                            init_token='<sos>',
                            eos_token='<eos>',
                            lower=True)
        TEXT.vocab = vectors.stoi

    def __getitem__(self, index):
        all_index = list(range(self.len))
        all_index.pop(index)
        all_index.pop(0)
        line = pd.read_csv(self.path, header=0, skiprows=all_index)
        line = ['<sos>']+line['Source'][0].split()+['<eos>']
        return self.embedder.tensorize(line)

    def __len__(self):
        return self.len

    def name(self):
        return 'AllSentencesDataset'

In [17]:
all_data = AllSentencesDataset('/data/SentenceEncoder/pytorch-seq2seq-master/train.csv', emb)

In [26]:
def collate_fn(batch):
    global device
    max_size = batch[0].size()
    max_lenght = max([s.size(0) for s in batch])
    embedding_dim = 300 # go to global
    padding_tensor = torch.tensor([0]*embedding_dim, dtype=torch.float).to(device) # go to global
    for sentence in range(len(batch)) :
        if max_lenght != batch[sentence].shape[0] :
            padder_tensor = torch.stack([padding_tensor]*(max_lenght-batch[sentence].shape[0]),dim=0).to(device)
            batch[sentence] = torch.cat((batch[sentence], padder_tensor), dim=0)
    return batch
        

In [40]:
dataloader = data.DataLoader(all_data, batch_size=24, shuffle=False, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=collate_fn,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None)

In [42]:
print(time.localtime())

for i in dataloader :
    print(len(i)) # longueur du batch
    print(i[0].shape)
    
print(time.localtime())

time.struct_time(tm_year=2020, tm_mon=3, tm_mday=20, tm_hour=8, tm_min=32, tm_sec=38, tm_wday=4, tm_yday=80, tm_isdst=0)
24
torch.Size([21, 300])
24
torch.Size([21, 300])
24
torch.Size([22, 300])
24
torch.Size([22, 300])
24
torch.Size([22, 300])
24
torch.Size([20, 300])
24
torch.Size([22, 300])
24
torch.Size([21, 300])


KeyboardInterrupt: 

In [ ]:
>>> t4d = torch.empty(3, 3, 4, 2)
>>> p1d = (1, 1) # pad last dim by 1 on each side
>>> out = F.pad(t4d, p1d, "constant", 0)  # effectively zero padding
>>> print(out.data.size())
torch.Size([3, 3, 4, 4])
>>> p2d = (1, 1, 2, 2) # pad last dim by (1, 1) and 2nd to last by (2, 2)
>>> out = F.pad(t4d, p2d, "constant", 0)
>>> print(out.data.size())
torch.Size([3, 3, 8, 4])
>>> t4d = torch.empty(3, 3, 4, 2)
>>> p3d = (0, 1, 2, 1, 3, 3) # pad by (0, 1), (2, 1), and (3, 3)
>>> out = F.pad(t4d, p3d, "constant", 0)
>>> print(out.data.size())
torch.Size([3, 9, 7, 3])

In [79]:
>>> from torch.nn.utils.rnn import pad_sequence
>>> a = torch.ones(25, 300)
>>> b = torch.ones(22, 300)
>>> c = torch.ones(15, 300)
>>> pad_sequence([a, b, c], padding_value=torch.ones(300)).size()
torch.Size([25, 3, 300])

RuntimeError: fill_ only supports 0-dimension value tensor but got tensor with 1 dimensions.

In [30]:
TEXT = torchtext.data.Field(tokenize='spacy',
                            init_token='<sos>',
                            eos_token='<eos>',
                            lower=True)

In [ ]:
TEXT.build_vocab(train_txt)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [31]:
import torchtext
from torchtext.data.utils import get_tokenizer
TEXT = torchtext.data.Field(tokenize=get_tokenizer("basic_english"),
                            init_token='<sos>',
                            eos_token='<eos>',
                            lower=True)
train_txt, val_txt, test_txt = torchtext.datasets.WikiText2.splits(TEXT)

downloading wikitext-2-v1.zip


wikitext-2-v1.zip: 100%|██████████| 4.48M/4.48M [00:03<00:00, 1.27MB/s]


extracting


In [32]:
train_txt

In [43]:
train_txt.examples[0].text

['<eos>',
 '=',
 'valkyria',
 'chronicles',
 'iii',
 '=',
 '<eos>',
 '<eos>',
 'senjō',
 'no',
 'valkyria',
 '3',
 '<unk>',
 'chronicles',
 '(',
 'japanese',
 '戦場のヴァルキュリア3',
 ',',
 'lit',
 '.',
 'valkyria',
 'of',
 'the',
 'battlefield',
 '3',
 ')',
 ',',
 'commonly',
 'referred',
 'to',
 'as',
 'valkyria',
 'chronicles',
 'iii',
 'outside',
 'japan',
 ',',
 'is',
 'a',
 'tactical',
 'role',
 '@-@',
 'playing',
 'video',
 'game',
 'developed',
 'by',
 'sega',
 'and',
 'media',
 '.',
 'vision',
 'for',
 'the',
 'playstation',
 'portable',
 '.',
 'released',
 'in',
 'january',
 '2011',
 'in',
 'japan',
 ',',
 'it',
 'is',
 'the',
 'third',
 'game',
 'in',
 'the',
 'valkyria',
 'series',
 '.',
 '<unk>',
 'the',
 'same',
 'fusion',
 'of',
 'tactical',
 'and',
 'real',
 '@-@',
 'time',
 'gameplay',
 'as',
 'its',
 'predecessors',
 ',',
 'the',
 'story',
 'runs',
 'parallel',
 'to',
 'the',
 'first',
 'game',
 'and',
 'follows',
 'the',
 'nameless',
 ',',
 'a',
 'penal',
 'military',
 'unit'